In [5]:
!python -m spacy download en_core_web_md

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-02 13:14:09.395425: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-02 13:14:09.395523: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-02 13:14:09.395541: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [1]:
# imports

import re
import numpy as np
import random
import spacy
import torch
import transformers
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from itertools import combinations
import networkx as nx



from transformers import AutoModelForQuestionAnswering, AutoTokenizer

import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [2]:
# Import the text from the saved file
with open('new_text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

nlp = spacy.load('en_core_web_md')
doc = nlp(text)
doc

Linear regression is a statistical method used to model the relationship between a dependent variable and one or more independent variables. It aims to find the best-fitting linear equation that describes the relationship between the variables.

The strengths of linear regression include its simplicity, interpretability, and ability to handle continuous variables. It can also be used for feature selection and to assess the importance of independent variables.

Weaknesses of linear regression include its sensitivity to outliers and the assumption of linearity between the independent variables and the dependent variable. It may also perform poorly if there are non-linear relationships between the variables.

Linear regression is commonly used in various fields, such as economics, social sciences, and engineering, to analyze and model the relationship between variables. It is also frequently used as a benchmark model for more complex machine learning algorithms.

Logistic regression is a 

In [3]:
# Defining question templates
templates = [
    "What is {subject} and what are the benefits of {subject}?",
    "Can you explain {subject}?",
    "How does {subject} work and given an application of {subject}?",
    "What are the applications of {subject}?"
]

In [4]:
# Define a function to extract keywords
def extract_keywords(doc, ngram_range=(1,3), min_freq=2):
    """Extract keywords from a Spacy Doc object using a given ngram_range and minimum frequency"""
    keywords = []
    for n in range(ngram_range[0], ngram_range[1]+1):
        ngrams = [doc[i:i+n] for i in range(len(doc)-n+1)]
        freq = {}
        for ngram in ngrams:
            if all(word.is_alpha and not word.is_stop and not word.is_punct and (word.pos_ == "NOUN" or word.pos_ == "PROPN") for word in ngram):
                freq[ngram.text.lower()] = freq.get(ngram.text.lower(), 0) + 1
        for ngram, f in freq.items():
            if f >= min_freq:
                keywords.append(ngram)
    return keywords

# this function gives us two and three words, keywords
def two_three_keywords(doc):

    # Extract unigrams, bigrams, and trigrams with a minimum frequency of 2
    keywords = set(extract_keywords(doc, ngram_range=(2,4), min_freq=1))

    keywords_two = set()
    keywords_three = set()

    for word in keywords:
        if len(word.strip().split(' ')) == 3:
            keywords_three.add(word)
        if len(word.strip().split(' ')) == 2:
            keywords_two.add(word)

    return list(keywords_two) + list(keywords_three)

In [5]:
# Creating questions

def getting_questions(doc, templates, n=5):

    questions = []
    keys = two_three_keywords(doc)
    keys = random.sample(keys, n)
    print(keys)

    i=0
    while i < 5:
        question = random.choice(templates)
        question = question.format(subject=keys[i])
        questions.append(question)
        i+=1

    return questions

In [10]:
"""

def generate_answers(questions, doc):
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
    model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

    max_chunk_size = 450
    chunked_text = [doc[i:i+max_chunk_size] for i in range(0, len(doc), max_chunk_size)]
    #print(chunked_text[0])

    answer = ""
    for question in questions:
        for text in chunked_text:

            # Tokenize the input
            inputs = tokenizer.encode_plus(question, str(text), add_special_tokens=True, return_tensors="pt")

            # Answer the question
            answer_start_scores, answer_end_scores = model(**inputs).values()
            answer_start = torch.argmax(answer_start_scores)
            answer_end = torch.argmax(answer_end_scores) + 1
            answer_chunk = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
            answer += answer_chunk.strip() + " "

        answer = re.sub(r'\[CLS\].*?\[SEP\]', '', answer)
        answer = re.sub(r'\[CLS\]', '', answer)

        # Print the answer
        print('Q:',question)
        print('Ans:',answer)
        print()
        answer = ''

questions = getting_questions(templates, 5)
generate_answers(questions, doc)

"""

'\n\ndef generate_answers(questions, doc):\n    # Load the tokenizer and model\n    tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")\n    model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")\n\n    max_chunk_size = 450\n    chunked_text = [doc[i:i+max_chunk_size] for i in range(0, len(doc), max_chunk_size)]\n    #print(chunked_text[0])\n\n    answer = ""\n    for question in questions:\n        for text in chunked_text:\n\n            # Tokenize the input\n            inputs = tokenizer.encode_plus(question, str(text), add_special_tokens=True, return_tensors="pt")\n\n            # Answer the question\n            answer_start_scores, answer_end_scores = model(**inputs).values()\n            answer_start = torch.argmax(answer_start_scores)\n            answer_end = torch.argmax(answer_end_scores) + 1\n            answer_chunk = tokenizer.convert_tokens_to_string(tokenizer.convert_

In [6]:
import csv
results = []

def generate_answers(questions, doc):
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
    model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

    max_chunk_size = 450
    chunked_text = [doc[i:i+max_chunk_size] for i in range(0, len(doc), max_chunk_size)]
    #print(chunked_text[0])
    print("Welcome to the technical interview!")
    candidate_name = input("What is your name? ")
    print("Hello, " + candidate_name + "! Let's get started.")

    answer = ""
    for question in questions:

        user_answer = input(f"Q: {question}\nAns: ")
        print()

        for text in chunked_text:

            # Tokenize the input
            inputs = tokenizer.encode_plus(question, str(text), add_special_tokens=True, return_tensors="pt")

            # Answer the question
            answer_start_scores, answer_end_scores = model(**inputs).values()
            answer_start = torch.argmax(answer_start_scores)
            answer_end = torch.argmax(answer_end_scores) + 1
            answer_chunk = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
            answer += answer_chunk.strip() + " "

        answer = re.sub(r'\[CLS\].*?\[SEP\]', '', answer)
        answer = re.sub(r'\[CLS\]', '', answer)

        results.append((question, answer, user_answer))

        # Print the answer
        # print('Q:',question)
        # print('Ans:',answer)
        # print()
        answer = ''

questions = getting_questions(doc, templates, 5)
generate_answers(questions, doc)

filename = "question_answers.csv"
with open(filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Question", "Model Answer", "User Answer"])
    for row in results:
        writer.writerow(row)

['language processing', 'series forecasting', 'linear discriminant', 'regression tasks', 'neural networks']
Welcome to the technical interview!
Hello, ! Let's get started.







In [7]:
# If we want to see the csv's path
import os
print(f"Results saved to {filename}")
print(f"The CSV file has been saved to: {os.getcwd()}/{filename}")

Results saved to question_answers.csv
The CSV file has been saved to: c:\Users\himan\ML\Spacy/question_answers.csv


In [15]:
# Reading csv file
import pandas as pd
df = pd.read_csv(r'C:\Users\himan\ML\Spacy\question_answers.csv')
print(df.head())

                                            Question  \
0  What is decision tree and what are the benefit...   
1  How does machine learning algorithms work and ...   
2                  Can you explain outcome variable?   
3  How does reinforcement learning work and given...   
4                       Can you explain mean square?   

                                        Model Answer User Answer  
0    improve the stability and robustness of the ...    Its good  
1  once the algorithm has learned these patterns,...         hey  
2   predicting the outcome variable    the varian...         hey  
3  training an agent to make decisions based on f...       g\\ji  
4  least squares   it squares the differences bet...          hi  


In [26]:
df['Model Answer'][0]

'  improve the stability and robustness of the model a tree - like model of decisions and their possible consequences, including chance event outcomes their ability to handle both categorical and continuous data, their ease of interpretation, and their ability to handle interactions between variables maximizes information gain or reduces impurity, ultimately creating a tree that accurately predicts the output variable for new inputs. pruning is a technique used in decision tree algorithms to reduce the size of a decision tree by removing some of its branches. the goal of pruning is to create a simpler decision tree that is less prone to overfitting and performs better on new, unseen data. in decision trees, overfitting occurs when the tree is too complex and has learned patterns in the training data that do not generalize well to new data. this can result in poor performance on new data and reduced accuracy. pruning can be done in two ways : pre - pruning and post - pruning. pre - prun

In [20]:
print("Welcome to the final round of interview!\n")
candidate_name = input("What is your name?")
print("\nHello, " + candidate_name + "! This is an integrated Tech + HR round.")
print("""
We will ask you a series of MCQ questions. 
Please provide your response to each question, and we will evaluate your answer on a scale of 1 to 10.
Every correct answer will award you +1 point and wrong will award you 0.
""")
print(candidate_name + ". Let's get started.")

Welcome to the final round of interview!


Hello, Abhinav! This is an integrated Tech + HR round.

We will ask you a series of MCQ questions. 
Please provide your response to each question, and we will evaluate your answer on a scale of 1 to 10.
Every correct answer will award you +1 point and wrong will award you 0.

Abhinav. Let's get started.


In [21]:
import json

#Load the JSON file
with open('mcq_questiosn.json', 'r') as f:
    questions = json.load(f)

# for item in questions:
#     print(item)

In [23]:
# json_str = json.dumps(questions, indent=4)
# print(json_str)

In [34]:
score = []
count = 0
# Iterate through the categories (Tech_MCQ and HR_MCQ)
for category in ['Tech_MCQ', 'HR_MCQ']:
    print(f"\n{category}\n")
    
    # Iterate through the questions in each category
    for question in questions[category]:
        print(question)
        for option in questions[category][question]:
            if option != "Answer":
                print(f"{option}: {questions[category][question][option]}")
        answer = input("Enter your answer (A/B/C/D): ")
        
        # Check if the answer is correct and update the score accordingly
        if answer == questions[category][question]["Answer"]:
            count += 1
            # print("\nCorrect!")
        # else:
            # print("\nIncorrect.")

        print()
        print()
    score.append(count)  
    count=0 
        
print(f"\nYour final score is {sum(score)}/{len(questions['HR_MCQ']) + len(questions['Tech_MCQ'])}")



Tech_MCQ

What is the difference between batch gradient descent and stochastic gradient descent in machine learning?
A: Batch gradient descent updates the parameters after computing the gradients on the entire dataset, while stochastic gradient descent updates the parameters after computing the gradients on a single data point.
B: Batch gradient descent updates the parameters after computing the gradients on a random subset of the dataset, while stochastic gradient descent updates the parameters after computing the gradients on the entire dataset.
C: Batch gradient descent updates the parameters after computing the gradients on a single data point, while stochastic gradient descent updates the parameters after computing the gradients on the entire dataset.
D: Batch gradient descent and stochastic gradient descent are the same thing.


What is the purpose of dropout regularization in deep learning?
A: To randomly select a subset of features to use in the model during training
B: To pre

### Scoring

In [ ]:
def HR_Scoring(user_answer_doc, hr_question_doc):
    
    # Calculate the cosine similarity between the answer and the HR question vectors
    similarity = cosine_similarity(user_answer_doc.vector.reshape(1,-1), hr_question_doc.vector.reshape(1,-1))

    return similarity[0][0]


def Tech_Scoring(user_answer_doc, model_answer_doc):
    
    # Calculate the cosine similarity between the answer and the HR question vectors
    similarity = cosine_similarity(user_answer_doc.vector.reshape(1,-1), model_answer_doc.vector.reshape(1,-1))

    return similarity[0][0]


# 50-30-20(Technical, HR, Resume)
def scoring(score, df):
    # Load the Spacy language model
    nlp = spacy.load('en_core_web_md')
    questions = df['Question'].values

    for i in range(0, len(questions)):

        # Tokenize and process the answer and the HR question
        user_answer_doc = nlp(df['User Answer'][i])
        model_answer_doc = nlp(df['Model Answer'][i])

        temp1 = HR_Scoring()
        temp2 = Tech_Scoring(user_answer_doc, model_answer_doc)

    tech_desc_score += temp2
    hr_desc_score += temp1

    # total score out of 10
    total_tech_score = score[0] + tech_desc_score
    total_hr_score = score[1] + hr_desc_score

    return total_tech_score*5, total_hr_score*3 , random.randrange(0, 10, 1)*2   # Vijay to complete the resume scoring

##### Fuzzy Inference System

In [ ]:
def FIS(score, df):
    # getting scores
    tech, hr, resume = scoring(score, df)

    # Define input variables
    resume = ctrl.Antecedent(np.arange(0, 21, 1), 'resume')
    technical = ctrl.Antecedent(np.arange(0, 51, 1), 'technical')
    HR = ctrl.Antecedent(np.arange(0, 31, 1), 'hr')

    # Define output variable
    score = ctrl.Consequent(np.arange(0, 101, 1), 'score')

    # Generate fuzzy membership functions for input variables
    resume['low'] = fuzz.trimf(resume.universe, [0, 0, 10])
    resume['medium'] = fuzz.trimf(resume.universe, [0, 10, 20])
    resume['high'] = fuzz.trimf(resume.universe, [10, 20, 20])
    technical['low'] = fuzz.trimf(technical.universe, [0, 0, 25])
    technical['medium'] = fuzz.trimf(technical.universe, [0, 25, 50])
    technical['high'] = fuzz.trimf(technical.universe, [25, 50, 50])
    HR['low'] = fuzz.trimf(HR.universe, [0, 0, 15])
    HR['medium'] = fuzz.trimf(HR.universe, [0, 15, 30])
    HR['high'] = fuzz.trimf(HR.universe, [15, 30, 30])

    # Generate fuzzy membership function for output variable
    score['low'] = fuzz.trimf(score.universe, [0, 0, 50])
    score['medium'] = fuzz.trimf(score.universe, [0, 50, 100])
    score['high'] = fuzz.trimf(score.universe, [50, 100, 100])

    # Define the rules for grading the resume
    rule1 = ctrl.Rule(resume['low'] & technical['low'] & HR['low'], score['low'])
    rule2 = ctrl.Rule(resume['low'] & technical['low'] & HR['medium'], score['low'])
    rule3 = ctrl.Rule(resume['low'] & technical['low'] & HR['high'], score['low'])
    rule4 = ctrl.Rule(resume['low'] & technical['medium'] & HR['low'], score['medium'])
    rule5 = ctrl.Rule(resume['low'] & technical['medium'] & HR['medium'], score['medium'])
    rule6 = ctrl.Rule(resume['low'] & technical['medium'] & HR['high'], score['medium'])
    rule7 = ctrl.Rule(resume['medium'] & technical['high'] & HR['low'], score['medium'])
    rule8 = ctrl.Rule(resume['medium'] & technical['high'] & HR['medium'], score['high'])
    rule9 = ctrl.Rule(resume['medium'] & technical['high'] & HR['high'], score['high'])
    rule10 = ctrl.Rule(resume['medium'] & technical['low'] & HR['low'], score['low'])
    rule11 = ctrl.Rule(resume['medium'] & technical['low'] & HR['medium'], score['low'])
    rule12 = ctrl.Rule(resume['medium'] & technical['low'] & HR['high'], score['medium'])
    rule13 = ctrl.Rule(resume['high'] & technical['medium'] & HR['low'], score['medium'])
    rule14 = ctrl.Rule(resume['high'] & technical['medium'] & HR['medium'], score['medium'])
    rule15 = ctrl.Rule(resume['high'] & technical['medium'] & HR['high'], score['medium'])
    rule16 = ctrl.Rule(resume['high'] & technical['high'] & HR['low'], score['medium'])
    rule17 = ctrl.Rule(resume['high'] & technical['high'] & HR['medium'], score['high'])
    rule18 = ctrl.Rule(resume['high'] & technical['high'] & HR['high'], score['high'])

    # Define control system and simulate
    score_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10, rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18])
    score_sim = ctrl.ControlSystemSimulation(score_ctrl)

    # Pass input values to simulation
    score_sim.input['resume'] = resume
    score_sim.input['technical'] = tech
    score_sim.input['hr'] = hr

    # Calculate output
    score_sim.compute()

    # Print result
    print("Candidates Score:", score_sim.output['score'])
